In [ ]:
import os
project_name = "reco-chef"; branch = "30music"; account = "sparsh-ai"
project_path = os.path.join('/content', project_name)

In [ ]:
if not os.path.exists(project_path):
    !pip install -U -q dvc dvc[gdrive]
    !cp -r /content/drive/MyDrive/git_credentials/. ~
    path = "/content/" + project_name; 
    !mkdir "{path}"
    %cd "{path}"
    !git init
    !git remote add origin https://github.com/"{account}"/"{project_name}".git
    !git pull origin "{branch}"
    !git checkout "{branch}"
else:
    %cd "{project_path}"

/content/reco-chef


In [ ]:
!git status

On branch 30music
Your branch is ahead of 'origin/30music' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   src/dataprep/30music.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	data/silver/
	dvc.lock
	dvc.yaml

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add . && git commit -m 'commit' && git push origin "{branch}"

usic 62521bb] commit
 4 files changed, 36 insertions(+), 1 deletion(-)
 create mode 100644 data/silver/.gitignore
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml
Counting objects: 13, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (10/10), done.
Writing objects: 100% (13/13), 1.32 KiB | 1.32 MiB/s, done.
Total 13 (delta 6), reused 0 (delta 0)
remote: Resolving deltas: 100% (6/6), completed with 4 local objects.
To https://github.com/sparsh-ai/reco-chef.git
   56541ac..62521bb  30music -> 30music


In [ ]:
!make setup

pip install -e .
Obtaining file:///content/reco-chef
  Running setup.py develop for recotut


In [ ]:
%reload_ext autoreload
%autoreload 2

## Prototyping

In [ ]:
!dvc pull ./data/bronze/30music/sessions_sample_10.parquet.snappy.dvc

  0% 0/1 [00:00<?, ?file/s{'info': ''}]Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Enter verification code: 4/1AX4XfWjOfKeUKWdEofAvX_7kBuG5YUYQ2DTWSntUmykmqUCVnCNx7f0sejU
Authentication successful.
Transferring:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
  0%|          |4e634fb3b7505d6fafc4293ffd2bc7     0.00/? [00:00<?,        ?B/s]
4e634fb3b7505d6fafc4293ffd2bc7:   0% 0.00/3.51M [00:00<?, ?B/s{'info': ''}]     
Checkout:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
.CZLTh7wvp2LDv2N8PS9okB.tmp:   0% 0.00/3.68M [00:00<?, ?it/s]
.CZLTh7wvp2LDv2N8PS9okB.tmp:   0% 0.00/3.68M [00:00<?, ?it/s{'info': ''}]
A       data/bronze/30music/sessions_sample_10.pa

In [ ]:
import os
import pandas as pd

from src.dataset import SessionDataset

In [ ]:
data_source_path = './data/bronze/30music/sessions_sample_10.parquet.snappy'
data_target_path = './data/silver/30music/'
filter_last_months = 1
filter_topk = 1000

In [ ]:
df = pd.read_parquet(data_source_path)
df.head()

,session_id,user_id,song_id,ts,playtime
0,1902204,4,16,1421163674,274
1,1902204,4,17,1421163948,250
2,1902204,4,18,1421164198,271
3,780919,10,60,1411009500,228
4,780919,10,61,1411014936,206


In [ ]:
sess_ds = SessionDataset(df)

In [ ]:
sess_ds.filter_by_time(filter_last_months)
sess_ds.convert_to_sequence(filter_topk)

In [ ]:
sess_ds.data.head()

,session_id,sequence,ts,user_id
0,357,"[793, 3489]",1421003874,4296
1,359,[1762],1421018535,4296
2,394,[1256],1421007470,30980
3,4127,"[1948, 1364, 2060, 1115, 6488, 2060]",1421416896,28117
4,6400,"[687, 1394]",1420807778,35247


In [ ]:
stats = sess_ds.get_stats()
print(stats)

Number of items: 1000
Number of users: 4165
Number of sessions: 6765
Session length:
	Average: 4.29
	Median: 3.0
	Min: 1
	Max: 148
Sessions per user:
	Average: 1.62
	Median: 1.0
	Min: 1
	Max: 13
Most popular items: [('443', 207), ('1065', 155), ('67', 146), ('2308', 138), ('658', 131)]


In [ ]:
sess_ds.last_session_out_split()
print("Train sessions: {} - Test sessions: {}".format(len(sess_ds.train), len(sess_ds.test)))

Train sessions: 2600 - Test sessions: 4165


In [ ]:
!mkdir -p ./data/silver/30music

In [ ]:
sess_ds.train.to_parquet(os.path.join(data_target_path, 'train.parquet.snappy'), compression='snappy')
sess_ds.test.to_parquet(os.path.join(data_target_path, 'test.parquet.snappy'), compression='snappy')

In [ ]:
!echo $data_target_path*.snappy

./data/silver/30music/test.parquet.snappy ./data/silver/30music/train.parquet.snappy


In [ ]:
!dvc -q add $data_target_path*.snappy
!dvc -q commit $data_target_path*.snappy
!dvc -q push $data_target_path*.snappy

In [ ]:
!rm -r ./data/silver/30music/*

## DVC Pipeline

In [ ]:
%%writefile params.yaml

prepare_30music:
  filter_last_months: 1
  filter_topk: 1000

Overwriting params.yaml


In [ ]:
%%writefile src/dataprep/30music.py

import os
import sys
import yaml
import pandas as pd

from src.dataset import SessionDataset

params = yaml.safe_load(open("params.yaml"))["prepare_30music"]

if len(sys.argv) != 2:
    sys.stderr.write("Arguments error. Usage:\n")
    sys.stderr.write("\tpython {dataprep}.py {data-source-filepath}\n")
    sys.exit(1)

filter_last_months = params['filter_last_months']
filter_topk = params['filter_topk']

data_source_path = str(sys.argv[1])
data_target_path = os.path.join("data", "silver", "30music")
data_target_path_train = os.path.join(data_target_path, 'train.parquet.snappy')
data_target_path_test = os.path.join(data_target_path, 'test.parquet.snappy')


def prepare_data(data_source_path,
                 filter_last_months, filter_topk,
                 data_target_path_train, data_target_path_test):
    
    df = pd.read_parquet(data_source_path)
    sess_ds = SessionDataset(df)

    sess_ds.filter_by_time(filter_last_months)
    sess_ds.convert_to_sequence(filter_topk)
    stats = sess_ds.get_stats()
    print(stats)
    sess_ds.last_session_out_split()
    print("Train sessions: {} - Test sessions: {}".format(len(sess_ds.train), len(sess_ds.test)))
    sess_ds.train.to_parquet(data_target_path_train, compression='snappy')
    sess_ds.test.to_parquet(data_target_path_test, compression='snappy')


os.makedirs(data_target_path, exist_ok=True)

prepare_data(data_source_path,
             filter_last_months, filter_topk,
             data_target_path_train, data_target_path_test)

Overwriting src/dataprep/30music.py


In [ ]:
!dvc run \
    -n prepare_30music \
    -p prepare_30music.filter_last_months,prepare_30music.filter_topk \
    -d src/dataprep/30music.py -d data/bronze/30music/sessions_sample_10.parquet.snappy \
    -o data/silver/30music \
    python src/dataprep/30music.py data/bronze/30music/sessions_sample_10.parquet.snappy

Running stage 'prepare_30music':
> python src/dataprep/30music.py data/bronze/30music/sessions_sample_10.parquet.snappy
Number of items: 1000
Number of users: 4165
Number of sessions: 6765
Session length:
	Average: 4.29
	Median: 3.0
	Min: 1
	Max: 148
Sessions per user:
	Average: 1.62
	Median: 1.0
	Min: 1
	Max: 13
Most popular items: [('443', 207), ('1065', 155), ('67', 146), ('2308', 138), ('658', 131)]
Train sessions: 2600 - Test sessions: 4165
Computing file/dir hashes (only done once)          |0.00 [00:00,      ?md5/s]
!
          |0.00 [00:00,       ?it/s]
                                    
!
          |0.00 [00:00,       ?it/s]
Transferring:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
d5ef5a4042482d29f581f56c967514.dir:   0% 0/159 [00:00<?, ?it/s]
d5ef5a4042482d29f581f56c967514.dir:   0% 0/159 [00:00<?, ?it/s{'info': ''}]
Creating 'dvc.yaml'
Adding stage 'prepare_30music' in 'dvc.yaml'
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run